## Import libraries

In [1]:
from copy import deepcopy
import re
import numpy as np
import pandas as pd

## Configurations

In [2]:
#pd.set_option('max_colwidth', 100)

## Load data

In [3]:
df_playstore = pd.read_csv('../data/googleplaystore.csv')

In general, to have an overview of the dataframe, I would print out the following information:
- data shape: the number of instances and features.
- several data rows: to have a sense of the values each data point may contain.
- data types of the columns.
- common statistics of the dataframe (using .describe() method).
- Missing-value status (using .isna() method).
- The number of unique values for each column.

In [4]:
print('df_playstore.shape:', df_playstore.shape)

df_playstore.shape: (10841, 13)


In [5]:
print('Some first rows of the data:')
display(df_playstore.head())

Some first rows of the data:


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [6]:
print('Column data-types:')
display(df_playstore.dtypes)

Column data-types:


App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [7]:
print('General statistics of the dataframe:')
display(df_playstore.describe(include='all'))

General statistics of the dataframe:


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
count,10841,10841,9367.000000,10841,10841,10841,10840,10841,10840,10841,10841,10833,10838
unique,9660,34,NaN,6002,462,22,3,93,6,120,1378,2832,33
top,ROBLOX,FAMILY,NaN,0,Varies with device,"1,000,000+",Free,0,Everyone,Tools,"August 3, 2018",Varies with device,4.1 and up
freq,9,1972,NaN,596,1695,1579,10039,10040,8714,842,326,1459,2451
mean,NaN,NaN,4.193338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,0.537431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,4.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,4.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print('Missing-value status:')
display(df_playstore.isna().sum())

Missing-value status:


App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [9]:
print('Number of unique values for each column:')
df_playstore.nunique()

Number of unique values for each column:


App               9660
Category            34
Rating              40
Reviews           6002
Size               462
Installs            22
Type                 3
Price               93
Content Rating       6
Genres             120
Last Updated      1378
Current Ver       2832
Android Ver         33
dtype: int64

#### Remove duplicates

In [10]:
# We don't need duplicated data points.
df_playstore = df_playstore.drop_duplicates(keep='first')
print('df_playstore.shape:', df_playstore.shape)

df_playstore.shape: (10358, 13)


#### Format the Last Updated column

In [11]:
# Show the rows that cannot be converted to datetime.
tmp_cast = pd.to_datetime(df_playstore['Last Updated']
                          , errors='coerce')
df_playstore[tmp_cast.isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [12]:
# shift the values to the right to correct this row.
row_missing_value = deepcopy(df_playstore.loc[10472])
row_missing_value[1:] = row_missing_value[1:].shift(periods=1)
df_playstore.loc[10472] = row_missing_value

# remove the temporary variable.
del row_missing_value

In [13]:
# this app belongs to the Lifestyle category
#   as manually checked on Google Store.
df_playstore.loc[10472, 'Category'] = 'LIFESTYLE'

In [14]:
# convert the column to datatime.
df_playstore['Last Updated'] = (
    pd.to_datetime(df_playstore['Last Updated'])
)

#### Rating

In [15]:
# convert to float.
df_playstore.Rating = df_playstore.Rating.astype(float)

#### Reviews column

In [16]:
# convert to int.
df_playstore['Reviews'] = df_playstore['Reviews'].astype(int)

#### Size column

In [17]:
# verify if all rows follow one of these 3 formats.
sum((df_playstore['Size'].str.match('\d+\.?\d*M').sum(),
     df_playstore['Size'].str.match('\d+\.?\d*k').sum(),
     df_playstore['Size'].str.match('Varies with device').sum()
    ))

10358

In [18]:
# copy the desired column to a new variable
#   and work on this new variable
#   to avoid unnecessary errors for the main data frame.
df_playstore_size = deepcopy(df_playstore['Size'])

# get the indexes of rows for each type of size.
index_size_M = df_playstore_size.str.endswith('M')
index_size_k = df_playstore_size.str.endswith('k')
index_size_unknown = df_playstore_size.str.match('Varies with device')

# replace the values of the copy.
df_playstore_size[index_size_M] = (
    df_playstore_size[index_size_M].apply(
        lambda x: float(x[:-1])
    )
)
df_playstore_size[index_size_k] = (
    df_playstore_size[index_size_k].apply(
        lambda x : float(x[:-1]) / 1024
    )
)
df_playstore_size[index_size_unknown] = np.nan

# apply the replaced value to the main data frame.
df_playstore['Size'] = df_playstore_size.astype(float)

# make a new column to emphasize the apps with
#   variable size.
df_playstore['Variable Size'] = (
    df_playstore['Size'].isna().astype(int)
)

# delete temporary variables.
del df_playstore_size
del index_size_M, index_size_k, index_size_unknown

#### Installs column

In [19]:
# see the buckets.
df_playstore['Installs'].value_counts()

1,000,000+        1488
10,000,000+       1132
100,000+          1129
10,000+           1033
1,000+             891
100+               710
5,000,000+         683
500,000+           517
50,000+            474
5,000+             469
10+                385
100,000,000+       369
500+               328
50,000,000+        272
50+                204
5+                  82
1+                  67
500,000,000+        61
1,000,000,000+      49
0+                  14
0                    1
Name: Installs, dtype: int64

In [20]:
# copy the desired column to a new variable
#   and work on this new variable
#   to avoid unnecessary errors for the main data frame.
df_playstore_installs = deepcopy(df_playstore.Installs)

# get the indexes of the rows that are 
#   not ready to be converted to int.
index_install_plus = df_playstore_installs.str.endswith('+')

# convert to int
df_playstore_installs[index_install_plus] = (
    df_playstore_installs[index_install_plus].apply(
        lambda x : int(x[:-1].replace(',', '')) + 1
    )
)

# apply the change to the main data frame
#   and rename the column to make it more clear.
df_playstore['Installs'] = df_playstore_installs.astype(int)
df_playstore = df_playstore.rename(
    columns={'Installs' : 'Installs at least'}
)

# delete the temporary variables.
del df_playstore_installs, index_install_plus

#### Type

In [21]:
# check for the rows with NaN Type.
df_playstore[df_playstore['Type'].isna()]

,App,Category,Rating,Reviews,Size,Installs at least,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Variable Size
9148,Command & Conquer: Rivals,FAMILY,NaN,0,NaN,0,NaN,0,Everyone 10+,Strategy,2018-06-28,Varies with device,Varies with device,1


In [22]:
# remove this row.
df_playstore = df_playstore.drop(9148)

In [23]:
# make a new dummy variable.
df_playstore['Is Free'] = (
    df_playstore.Type == 'Free'
).astype(int)

# remove the old row.
df_playstore = df_playstore.drop(['Type'], axis=1)

#### Price

In [24]:
# verify if all prices follow one of these two formats.
print (sum(df_playstore.Price.str.match('^\$\d+\.?\d*$'))
       + sum(df_playstore.Price == '0')
      )

10357


In [25]:
# copy the desired column to a new variable
#   and work on this new variable
#   to avoid unnecessary errors for the main data frame.
df_playstore_price = deepcopy(df_playstore.Price)

# get the indexes of the rows that are 
#   not ready to be converted to float.
index_price_dollar = (
    df_playstore.Price.str.match('^\$\d+\.?\d*$')
)

# do the conversion.
df_playstore_price[index_price_dollar] = (
    df_playstore_price[index_price_dollar].str[1:].astype(float)
)

# apply the change to the main data frame.
df_playstore.Price = df_playstore_price

# delete temporary variables
del df_playstore_price, index_price_dollar

#### Content Rating

In [27]:
# list the tags and their value counts.
df_playstore['Content Rating'].value_counts()

Everyone           8383
Teen               1146
Mature 17+          447
Everyone 10+        376
Adults only 18+       3
Unrated               2
Name: Content Rating, dtype: int64

In [28]:
# map the values of Content Rating to numbers
#   and store them in another column.
df_playstore['Minimum Age'] = df_playstore['Content Rating'].map(
    {'Everyone' : 0,
     'Everyone 10+' : 10,
     'Teen' : 13,
     'Mature 17+' : 17,
     'Adults only 18+' : 18,
     'Unrated' : np.nan
    }
)

# remove the old Content Rating,
# we already have Minimum Age as a replacement.
df_playstore = df_playstore.drop(['Content Rating'], axis=1)

#### Android Ver

In [29]:
# list the rows with NaN Android Ver.
df_playstore[df_playstore['Android Ver'].isna()]

,App,Category,Rating,Reviews,Size,Installs at least,Price,Genres,Last Updated,Current Ver,Android Ver,Variable Size,Is Free,Minimum Age
4453,[substratum] Vacuum: P,PERSONALIZATION,4.4,230,11.0,1001,1.49,Personalization,2018-07-20,4.4,NaN,0,0,0.0
4490,Pi Dark [substratum],PERSONALIZATION,4.5,189,2.1,10001,0,Personalization,2018-03-27,1.1,NaN,0,1,0.0


So there is only a small number of apps that were halted from updating to be competible with newer android versions.

For these apps, we may create a new column, e.g. Is Still Maintained.

In [30]:
# list the value counts
df_playstore['Android Ver'].value_counts()

4.1 and up            2379
4.0.3 and up          1451
4.0 and up            1338
Varies with device    1220
4.4 and up             894
2.3 and up             643
5.0 and up             546
4.2 and up             387
2.3.3 and up           279
2.2 and up             239
3.0 and up             237
4.3 and up             235
2.1 and up             133
1.6 and up             116
6.0 and up              58
7.0 and up              42
3.2 and up              36
2.0 and up              32
5.1 and up              22
1.5 and up              20
4.4W and up             11
3.1 and up              10
2.0.1 and up             7
8.0 and up               6
7.1 and up               3
1.0 and up               2
4.0.3 - 7.1.1            2
5.0 - 8.0                2
4.1 - 7.1.1              1
5.0 - 6.0                1
5.0 - 7.1.1              1
2.2 - 7.1.1              1
7.0 - 7.1.1              1
Name: Android Ver, dtype: int64

Note that there is a value '4.4W and up' - 4.4W is the 4.4 with extended supports for Wearables.

In [31]:
# function to extract the major android version from text
# the major version starts from the beginning and
#   ends before the first dot (.).
# NaN is returned if cannot extract.
def get_android_major_version(text):
    try:
        ver = int(text[:text.find('.')])
        return ver
    except:
        return np.nan

df_playstore['Android Major Ver From'] = (
    df_playstore['Android Ver'].apply(get_android_major_version)
)

In [32]:
df_playstore['Variable Android Ver'] = (
    (df_playstore['Android Ver'].str
         .match('^Varies with device$') == True
    ).astype(int)
)

In [33]:
# The apps whose Android ver end with 'and up'
#   or equals to 'Varies with device'
#   are considered still maintained
df_playstore['Is Still Maintained'] = (
    df_playstore['Android Ver'].str.endswith('and up') | 
    df_playstore['Android Ver'].str.match(r'^Varies with device$')
)

In [34]:
# Remove the no-longer-wanted column.
df_playstore = df_playstore.drop(['Android Ver'], axis=1)

#### Current Ver

In [35]:
# prepare a function that can recognize if
#   a Current Ver value indicate an updated version
#   or initial version.
# if the input value is 'Varies with device'
#   that means the app has been updated.
# if the input does not follow the standard
#   return NaN
# if the input follows the standard then
#   extract the version and check if it is
#   1 or 1.0 or 1.0.0 etc.
app_ver_std_re = re.compile(r'(?i)(?:v|version)?:? ?(\d.*)')
first_version_re = re.compile(r'1(\.0)*$')

def recognized_has_updated(current_ver):
    if current_ver == 'Varies with device':
        return 1
    match = re.match(app_ver_std_re, current_ver)
    if match:
        version = match.group(1)
        if re.match(first_version_re, version):
            return 0
        else:
            return 1
    else:
        return np.nan

# apply 
df_playstore['Has Ever Updated'] = (
    df_playstore['Current Ver'].apply(
        lambda x : recognized_has_updated(str(x))
    )
)

In [36]:
# make a column to indicate if
#   the app has different versions
#   for different devices.
df_playstore['Variable App Ver'] = (
    df_playstore['Current Ver'].str
    .match(r'^Varies with device$')
    .astype(float)
)

In [37]:
# write a function to extract the major version 
# the regex used this time is very similar
#   to that of the recognized_has_updated,
#   the only difference is on the capturing group.
extract_major_ver_re = re.compile(r'(?i)(?:v|version)?:? ?(\d).*')

def extract_app_major_version(current_ver):
    match = re.match(extract_major_ver_re, current_ver)
    if match:
        major_version = match.group(1)
        return major_version
    return np.nan

# apply
df_playstore['Major App Ver'] = (
    df_playstore['Current Ver'].apply(
        lambda x : extract_app_major_version(str(x))
    )
)

In [38]:
# Current Ver has fulfilled its role.
df_playstore = df_playstore.drop(['Current Ver'], axis=1)

#### Category and Genres

- No NaNs here.
- An app may be associated with multiple genres.

In [39]:
# One-hot encode Category
df_playstore = (
    pd.get_dummies(
        df_playstore, columns=['Category'], prefix='Cat'
    )
)

In [40]:
# cast Genres to string type
df_playstore.Genres = df_playstore.Genres.astype(str)

# split the genres of each app
df_stacked_genres = (
    df_playstore.Genres.str
    .split(';', expand=True).stack()
)

# apply one-hot encoding
df_genres_dummies = (
    pd.get_dummies(df_stacked_genres, prefix='gen')
    .groupby(level=0).sum()
)

# join the one-hot result into the main data frame
df_playstore = (df_playstore.merge(df_genres_dummies, 
                                   left_index=True, 
                                   right_index=True)
               ).drop(['Genres'], axis=1)

# delete temporary variables
del df_stacked_genres, df_genres_dummies

-------------------------------------------------------------------

We have done with this data cleaning.

In [41]:
df_playstore.head()

,App,Rating,Reviews,Size,Installs at least,Price,Last Updated,Variable Size,Is Free,Minimum Age,...,gen_Social,gen_Sports,gen_Strategy,gen_Tools,gen_Travel & Local,gen_Trivia,gen_Video Players & Editors,gen_Weather,gen_Word,gen_nan
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,19.0,10001,0,2018-01-07,0,1,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,3.9,967,14.0,500001,0,2018-01-15,0,1,0.0,...,0,0,0,0,0,0,0,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,8.7,5000001,0,2018-08-01,0,1,0.0,...,0,0,0,0,0,0,0,0,0,0
3,Sketch - Draw & Paint,4.5,215644,25.0,50000001,0,2018-06-08,0,1,13.0,...,0,0,0,0,0,0,0,0,0,0
4,Pixel Draw - Number Art Coloring Book,4.3,967,2.8,100001,0,2018-06-20,0,1,0.0,...,0,0,0,0,0,0,0,0,0,0
